In [4]:
%%bash
python --version
java -version
#scala -version

Python 3.9.7


java version "1.8.0_311"
Java(TM) SE Runtime Environment (build 1.8.0_311-b11)
Java HotSpot(TM) 64-Bit Server VM (build 25.311-b11, mixed mode)


In [5]:
java.sun.com
http://java.sun.com/products/autodl/j2se
/Library/Internet Plug-Ins/JavaAppletPlugin.plugin/Contents/Home/bin/java

SyntaxError: invalid syntax (2910056770.py, line 2)

In [6]:
import pyspark

In [7]:
from pyspark import SparkContext
from pyspark.sql import SQLContext

In [8]:
sc = SparkContext()
sc

21/11/27 23:07:15 WARN Utils: Your hostname, srimac.local resolves to a loopback address: 127.0.0.1; using 192.168.1.10 instead (on interface en0)
21/11/27 23:07:15 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/11/27 23:07:16 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
21/11/27 23:07:18 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
21/11/27 23:07:18 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


<SparkContext master=local[*] appName=pyspark-shell>

In [9]:
from pyspark.sql import SparkSession

spark = SparkSession \
        .builder \
        .appName("Analyzing Movies Dataset") \
        .getOrCreate()

#### Load the dataset

Both datasets have been sourced from https://www.kaggle.com/rounakbanik/the-movies-dataset. <br />
The first file (ratings.csv) contains 26 million ratings from 270,000 users on all the 45,000 movies but here we are considering only 2 million ratings for this demo. <br />
The second file (movies_metadata.csv) includes information about budget, revenue, release dates, languages, production countries etc. for each movie.

In [10]:
movie_ratings = spark.read\
               .format("csv")\
               .option("sep", ",")\
               .option("header", "true")\
               .load("../datasets/ratings.csv")

In [11]:
movie_ratings.printSchema()

root
 |-- userId: string (nullable = true)
 |-- movieId: string (nullable = true)
 |-- rating: string (nullable = true)
 |-- timestamp: string (nullable = true)



In [12]:
movie_metadata = spark.read\
                         .format("csv")\
                         .option("sep", ",")\
                         .option("header", "true")\
                         .load("../datasets/movies_metadata.csv")

In [13]:
movie_metadata.printSchema()

root
 |-- adult: string (nullable = true)
 |-- belongs_to_collection: string (nullable = true)
 |-- budget: string (nullable = true)
 |-- genres: string (nullable = true)
 |-- homepage: string (nullable = true)
 |-- id: string (nullable = true)
 |-- imdb_id: string (nullable = true)
 |-- original_language: string (nullable = true)
 |-- original_title: string (nullable = true)
 |-- overview: string (nullable = true)
 |-- popularity: string (nullable = true)
 |-- poster_path: string (nullable = true)
 |-- production_companies: string (nullable = true)
 |-- production_countries: string (nullable = true)
 |-- release_date: string (nullable = true)
 |-- revenue: string (nullable = true)
 |-- runtime: string (nullable = true)
 |-- spoken_languages: string (nullable = true)
 |-- status: string (nullable = true)
 |-- tagline: string (nullable = true)
 |-- title: string (nullable = true)
 |-- video: string (nullable = true)
 |-- vote_average: string (nullable = true)
 |-- vote_count: string (nu

In [14]:
movie_ratings.count(),movie_metadata.count()

(1999999, 45572)

#### Distinct number of Id

In [15]:
movie_metadata.select('id')\
                 .distinct()\
                 .count()

45470

#### Drop columns which are not necessary for our analysis

In [16]:
movie_metadata = movie_metadata.drop(
                        'belongs_to_collection', 
                        'genres',
                        'homepage',
                        'imdb_id',
                        'overview',
                        'poster_path',
                        'production_companies',
                        'production_countries',
                        'spoken_languages',
                        'tagline')
movie_metadata

DataFrame[adult: string, budget: string, id: string, original_language: string, original_title: string, popularity: string, release_date: string, revenue: string, runtime: string, status: string, title: string, video: string, vote_average: string, vote_count: string]

#### View the schema of the dataframes
Though many numeric columns are currently formatted as strings, we'll adjust their types when we need to later on

In [17]:
movie_metadata.printSchema()

root
 |-- adult: string (nullable = true)
 |-- budget: string (nullable = true)
 |-- id: string (nullable = true)
 |-- original_language: string (nullable = true)
 |-- original_title: string (nullable = true)
 |-- popularity: string (nullable = true)
 |-- release_date: string (nullable = true)
 |-- revenue: string (nullable = true)
 |-- runtime: string (nullable = true)
 |-- status: string (nullable = true)
 |-- title: string (nullable = true)
 |-- video: string (nullable = true)
 |-- vote_average: string (nullable = true)
 |-- vote_count: string (nullable = true)



#### Drop the invalid data from the dataframes

In [18]:
movie_metadata = movie_metadata.dropna()
movie_ratings = movie_ratings.dropna()

#### The size of movie_metadata has reduced

In [19]:
movie_ratings.count(),movie_metadata.count()

(1999999, 43492)

#### Cast the numeric columns as float
These are currently strings

In [20]:
for col_name in ["budget", "revenue", "vote_average", "popularity"]:
    movie_metadata = movie_metadata.withColumn(col_name, 
                                               movie_metadata[col_name].cast("float"))
    
movie_metadata.printSchema()

root
 |-- adult: string (nullable = true)
 |-- budget: float (nullable = true)
 |-- id: string (nullable = true)
 |-- original_language: string (nullable = true)
 |-- original_title: string (nullable = true)
 |-- popularity: float (nullable = true)
 |-- release_date: string (nullable = true)
 |-- revenue: float (nullable = true)
 |-- runtime: string (nullable = true)
 |-- status: string (nullable = true)
 |-- title: string (nullable = true)
 |-- video: string (nullable = true)
 |-- vote_average: float (nullable = true)
 |-- vote_count: string (nullable = true)



#### We are interested in positive budget and revenue values

In [21]:
movie_metadata = movie_metadata[(movie_metadata.budget > 0) \
                                & (movie_metadata.revenue > 0)]

In [22]:
movie_metadata.count()

4981

In [23]:
movie_metadata.select("budget", "revenue").show()

+---------+------------+
|   budget|     revenue|
+---------+------------+
|    3.0E7|3.73554048E8|
|    6.5E7|2.62797248E8|
|    6.0E7|1.87436816E8|
|    3.5E7| 6.4350172E7|
|    5.8E7|3.52194048E8|
|    6.2E7|1.07879496E8|
|    4.4E7| 1.3681765E7|
|    9.8E7| 1.0017322E7|
|    5.2E7|1.16112376E8|
|   1.65E7|      1.35E8|
|4000000.0|   4300000.0|
|    3.0E7|2.12385536E8|
|    6.0E7| 3.5431112E7|
|    5.0E7| 3.0303072E7|
|3600000.0|      4.98E7|
|    1.2E7|      2.74E7|
|   2.95E7|    1.6884E8|
|    3.0E7|2.54134912E8|
|    1.1E7| 3.9363636E7|
|    1.8E7| 1.2219592E8|
+---------+------------+
only showing top 20 rows



In [24]:
movie_metadata = movie_metadata.withColumn("profit",
                                           (movie_metadata["revenue"] - movie_metadata["budget"]))

In [25]:
movie_metadata.select("profit").sort(movie_metadata["profit"].desc())\
                               .show()

+------------+
|      profit|
+------------+
|2.55096525E9|
|1.82322355E9|
|1.64503424E9|
|1.36352883E9|
|1.31624934E9|
|1.29955789E9|
|1.21699994E9|
|1.12540365E9|
|1.12421901E9|
| 1.1028864E9|
|1.08273101E9|
|1.02488896E9|
|   1.01544E9|
|  9.887648E8|
| 9.4006336E8|
| 9.2874694E8|
| 9.0856102E8|
| 9.0330445E8|
| 8.9476186E8|
| 8.7378419E8|
+------------+
only showing top 20 rows



In [26]:
movie_metadata.printSchema()

root
 |-- adult: string (nullable = true)
 |-- budget: float (nullable = true)
 |-- id: string (nullable = true)
 |-- original_language: string (nullable = true)
 |-- original_title: string (nullable = true)
 |-- popularity: float (nullable = true)
 |-- release_date: string (nullable = true)
 |-- revenue: float (nullable = true)
 |-- runtime: string (nullable = true)
 |-- status: string (nullable = true)
 |-- title: string (nullable = true)
 |-- video: string (nullable = true)
 |-- vote_average: float (nullable = true)
 |-- vote_count: string (nullable = true)
 |-- profit: float (nullable = true)



21/11/28 10:45:03 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 9999635 ms exceeds timeout 120000 ms
21/11/28 10:45:03 WARN SparkContext: Killing executors is not supported by current scheduler.


In [ ]:
from pyspark.sql.functions import udf

#### We are spliting the date column and from here we will take only year and save it in another column.

In [ ]:
year_extract_udf = udf(lambda date: date.split('-')[0])

movie_metadata = movie_metadata.withColumn(
                    "release_year",
                    year_extract_udf(movie_metadata.release_date)
)

#### Drop the date column

In [ ]:
movie_metadata =movie_metadata.drop("release_date")

In [ ]:
movie_metadata.s

#### Filter the dataset to movies released since 1995 

In [ ]:
movies_1995 = movie_metadata.filter(movie_metadata.release_year >= 1995)

In [ ]:
movies_1995.count()

#### Check the number of distinct languages in this filtered dataset

In [ ]:
movies_1995.select(movies_1995.original_language)\
       .distinct()\
       .count()

#### Check average popularity and profits for movies in each language

In [ ]:
movies_1995 = movies_1995.groupBy("original_language")\
                         .agg({"popularity": "avg",
                               "profit": "avg"
                              })

In [ ]:
movies_1995 =movies_1995.withColumnRenamed("avg(popularity)", "popularity")\
                        .withColumnRenamed("avg(profit)", "profit")

#### How popular and profitable are movies in each language?
* English, German, Spanish films are popular and profitable
* Vietnamese films are neither profitable nor popular
* Romanian films are mildly popular but unprofitable

In [ ]:
movies_1995.show()

#### Define your own metric to calculate how "good" a movie is
We scale the popularity and profitabiltiy values

In [ ]:
weight_popularity = 10
weight_profit=0.000001

In [ ]:
movies = movies_1995.withColumn("score",
                                    (movies_1995.popularity * weight_popularity + \
                                     movies_1995.profit * weight_profit))

In [ ]:
movies.show()

#### We are interested in movies with a score greater than 50
English, Japanese and Chinese movies top our list

In [ ]:
movies = movies.filter(movies.score > 50)\
                   .sort(movies.score.desc())
    
movies.show(15)

In [ ]:
from pyspark.sql.functions import broadcast

In [ ]:
movie_ratings.count(),movie_metadata.count()

### Broadcast & Join

* Broadcast the smaller dataframe so it is available on all cluster machines
* The data should be small enough so it is held in memory
* All nodes in the cluster distribute the data as fast as they can so overall computation is faster

In [ ]:
movies_joined = movie_ratings.select('movieId',
                                     'rating')\
                             .join(broadcast(movie_metadata),
                                   movie_ratings.movieId == movie_metadata.id, 'inner')

In [ ]:
movies_joined.columns

In [ ]:
movies_joined = movies_joined.drop(
                                     'adult',
                                     'id',
                                     'budget',
                                     'original_title',
                                     'popularity',
                                     'revenue',
                                     'runtime',
                                     'status',
                                     'video',
                                     'vote_average',
                                     'vote_count',
                                     'profit',
                                     'release_year')
movies_joined

In [ ]:
movies_joined.show(truncate = False)

In [ ]:
movies_joined.groupBy(["movieID", "title"])\
             .agg({"rating": "avg"}).show()

In [ ]:
movies_joined.select("rating").describe().show()

### Accumulators
* Shared variables which are updated by processes running across multiple nodes

In [ ]:
terrible_count = spark.sparkContext.accumulator(0)
subpar_count = spark.sparkContext.accumulator(0)
average_count = spark.sparkContext.accumulator(0)
good_count = spark.sparkContext.accumulator(0)

#### Get movie count by ratings

In [ ]:
def count_movie_by_rating(row):
    rating = float(row.rating)
    
    if (rating <= 2.0 ):
        terrible_count.add(1)
    elif (rating <= 3.0 and rating > 2.0 ):
        subpar_count.add(1)
    elif (rating <= 4.0 and rating > 3.0 ):
        average_count.add(1)
    elif (rating > 4.0) :
        good_count.add(1)

In [ ]:
movies_joined.foreach(lambda x: count_movie_by_rating(x))

In [ ]:
print("Terrible movies: ", terrible_count.value)
print("Sub-par movies: ", subpar_count.value)
print("Average movies: ", average_count.value)
print("Good movies: ", good_count.value)

### Save the dataframe to a file

#### Save to a CSV file
Saves the data into a directory. You could also give a path in HDFS

In [ ]:
movies_1995.write\
           .option("header","true")\
           .csv("movies_1995_csv")

#### Coalesce data into one file
Data from all partitions will coalesce into a single list

In [ ]:
movies_1995.coalesce(1)\
           .write\
           .option("header","true")\
           .csv("movies_1995_coalesce_csv")

#### Save data as a JSON file

In [ ]:
movies_1995.coalesce(1)\
           .write\
           .json("movie_overall_json")